In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import mysql.connector
from mysql.connector import Error

In [2]:
query_return_table = 'SELECT t.Conv_Date AS "Observation", COUNT(Conv_ID) AS "Total_Orders", COUNT(CASE WHEN t.Customer_Type = "Returning" THEN Conv_ID END) AS "Returning_Orders", COUNT(CASE WHEN t.Customer_Type = "Returning" THEN Conv_ID END)/COUNT(Conv_ID) AS "Repurchase Rate" FROM (SELECT fo.Customer_ID, fo.Conv_Date, fo.Conv_ID, CASE WHEN Conv_Date = first_orders.first_date THEN "New" ELSE "Returning" END AS "Customer_Type" FROM etl_site_traffic.conversions_backend fo JOIN ( SELECT Customer_ID, MIN(Conv_Date) AS "first_date" FROM etl_site_traffic.conversions_backend cb WHERE Account = "Paula" AND Market = "DE" GROUP BY 1) AS first_orders ON first_orders.Customer_ID = fo.Customer_ID WHERE Account = "Paula" AND Market = "DE") AS t GROUP BY 1;'

query_rfm_table = 'SELECT Customer_ID, COUNT(Conv_ID) AS "frequency", DATEDIFF(CURDATE(),MAX(Conv_Date)) AS "recency", AVG(Revenue_excl_vat) AS "monetary_value" FROM etl_site_traffic.conversions_backend cb WHERE Account = "Paula" AND Market = "DE" GROUP BY 1;'

In [4]:
try:
    connection = mysql.connector.connect(host='attribution-system-fsg-new.cob86lv75rzo.eu-west-1.rds.amazonaws.com',
                        database='etl_site_traffic',
                        user='fsg',
                        password='Attribution3.0')
    if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("Your connected to database: ", record)
            df_return = pd.read_sql(query_return_table,con=connection)
            df_rfm = pd.read_sql(query_rfm_table,con=connection)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

Connected to MySQL Server version  8.0.16
Your connected to database:  ('etl_site_traffic',)
MySQL connection is closed


In [5]:
df_return.shape, df_rfm.shape

((4152, 4), (280151, 4))

In [6]:
df_rfm

,Customer_ID,frequency,recency,monetary_value
0,596a078be39d2a041cc268a585920ce1,4,544,89.222500
1,d5260d3f20bb53c528f3bcadd57fe3bf,3,607,62.296667
2,3d9a9ab7d093bbdf0f8ccda6cc027459,6,312,55.371667
3,0d4daf40508e79fe5071a59f89dca1c9,17,946,71.098235
4,e247168c13ffaa8d6cbffa30d90a3eff,30,255,190.096667
...,...,...,...,...
280146,3de604dcd04e4d00c40bce05c25a1f66,1,69,38.030000
280147,6dd976418d08a8c8ccc86076b23e2272,1,69,16.890000
280148,eadebfd1417d4f8aa42a746c0c281b37,1,74,0.000000
280149,56f3a582e2a888331c179cf065e4a666,1,281,0.000000


In [8]:
# Purchase Frequency
purchase_frequency=sum(df_rfm['frequency'])/df_rfm.shape[0]

# Repeat Rate
repeat_rate=df_rfm[df_rfm.frequency > 1].shape[0]/df_rfm.shape[0]

# Churn Rate
churn_rate=1-repeat_rate

# Customer Value
df_rfm['CLV']=(df_rfm['monetary_value']*purchase_frequency)/churn_rate

In [9]:
df_rfm.quantile([.2, .4, .6, .8], axis=0)

,frequency,recency,monetary_value,CLV
0.2,1.0,272.0,22.020000,101.106510
0.4,1.0,361.0,32.690000,150.098629
0.6,2.0,567.0,47.440000,217.824379
0.8,3.0,1080.0,69.703333,320.048172


In [10]:
purchase_frequency,repeat_rate,churn_rate

(2.6674971711684057, 0.4190454433501933, 0.5809545566498067)

In [11]:
df_rfm.sample(5)

,Customer_ID,frequency,recency,monetary_value,CLV
273283,a71f7d46fb09ee7768962229f675613b,1,223,48.3600,222.048630
69788,ab890657f4db983df9d2f45f00726fbf,3,1126,50.1800,230.405299
201655,b6a407bee24b7e353e80718868c36a9e,2,224,55.8600,256.485452
150850,01d6e45f564628bd932532e2115aa989,4,226,87.3825,401.223416
1099,86ba6de92134c35fdff591cc2f937410,1,3551,18.4000,84.485004


In [18]:
df_rfm.recency.median()

133.0

## OTHER RFM STUFF

In [54]:
rfm['r_score'] = 0
for d in range(len(rfm)):
    if rfm.recency[d] < 60:
        rfm['r_score'][d] = 5
    elif rfm.recency[d] < 190:
        rfm['r_score'][d] = 4
    elif rfm.recency[d] < 400:
        rfm['r_score'][d] = 3
    elif rfm.recency[d] < 610:
        rfm['r_score'][d] = 2
    else:
        rfm['r_score'][d] = 1


In [55]:
rfm['f_score'] = 0
for o in range(len(rfm)):
    if rfm.frequency[o] == 1:
        rfm['f_score'][o] = 1
    elif rfm.frequency[o] == 2:
        rfm['f_score'][o] = 2
    elif rfm.frequency[o] == 3:
        rfm['f_score'][o] = 3
    elif rfm.frequency[o] == 4:
        rfm['f_score'][o] = 4
    else:
        rfm['f_score'][o] = 5
    

In [56]:
rfm['m_score'] = 0
for o in range(len(rfm)):
    if rfm.monetary_value[o] < 33:
        rfm['m_score'][o] = 1
    elif rfm.monetary_value[o] < 43:
        rfm['m_score'][o] = 2
    elif rfm.monetary_value[o] < 50:
        rfm['m_score'][o] = 3
    elif rfm.monetary_value[o] < 61:
        rfm['m_score'][o] = 4
    else:
        rfm['m_score'][o] = 5

In [58]:
rfm['score_conc'] = ''
for s in range(len(rfm)):
    rfm['score_conc'][s] = str(rfm['r_score'][s])+str(rfm['f_score'][s])+str(rfm['m_score'][s])

In [59]:
rfm.head()

,email,recency,frequency,monetary_value,CLV,r_score,f_score,m_score,score_conc
0,003ppcs4s2vcj0q@marketplace.amazon.de,734,3,40.150000,312.997351,1,3,2,132
1,009g57nc9rg76dy@marketplace.amazon.de,32,11,45.452727,354.335821,5,5,3,553
2,00j89nxrhbft07k@marketplace.amazon.de,1060,1,44.810000,349.325313,1,1,3,113
3,00jx5l3sntnj5p1@marketplace.amazon.de,847,1,23.830000,185.771529,1,1,1,111
4,00sk0ls66hd2c5x@marketplace.amazon.de,958,2,27.050000,210.873682,1,2,1,121


In [69]:
rfm['clv_score'] = ''
for d in range(len(rfm)):
    if rfm.CLV[d] < 215:
        rfm['clv_score'][d] = "Low"
    elif rfm.CLV[d] < 277:
        rfm['clv_score'][d] = "Mid-Low"
    elif rfm.CLV[d] < 320:
        rfm['clv_score'][d] = "Mid"
    elif rfm.CLV[d] < 386:
        rfm['clv_score'][d] = "Mid-High"
    elif rfm.CLV[d] >= 386:
         rfm['clv_score'][d] = "High"


In [62]:
promising = rfm[rfm['score_conc'].isin(["525", "524", "523", "522", "521", "515", "514", "513", "425", "424", "413", "414", "415", "315", "314", "313"])].loc[:,['email']]

In [66]:
rfm[rfm['score_conc'].isin(["525", "524", "523", "522", "521", "515", "514", "513", "425", "424", "413", "414", "415", "315", "314", "313"])]

,email,recency,frequency,monetary_value,CLV,r_score,f_score,m_score,score_conc,clv_score
6,00w4f6c1l0dr8wr@marketplace.amazon.de,39,2,36.400,283.763476,5,2,2,522,Mid
7,00y6n4lv9yxzn94@marketplace.amazon.de,343,1,57.710,449.889841,3,1,4,314,High
51,05xzx8jgpk2xdy5@marketplace.amazon.de,115,1,52.480,409.118331,4,1,4,414,High
54,06b2pyv78fyqq3d@marketplace.amazon.de,40,2,49.485,385.770210,5,2,3,523,Mid-High
97,0d79hym02d70vj2@marketplace.amazon.de,196,1,53.470,416.836074,3,1,4,314,High
...,...,...,...,...,...,...,...,...,...,...
23468,zt03hsjr6gx8hqv@marketplace.amazon.de,197,1,58.810,458.465111,3,1,4,314,High
23471,ztbr3mwpwl6zfr2@marketplace.amazon.de,266,1,46.050,358.991980,3,1,3,313,Mid-High
23481,zuechter1@gmx.de,10,1,102.800,801.397950,5,1,5,515,High
23506,zwinger_von_itzenplitz@gmx.de,256,1,108.810,848.250106,3,1,5,315,High


In [63]:
promising.shape

(1531, 1)

In [70]:
clv_relevant = rfm[rfm['clv_score'].isin(["Mid-High", "High"])].loc[:,['email']]
clv_relevant.shape

(8964, 1)

In [68]:
rfm.shape # but in powr bi there are only 15,466 customers

(23530, 10)

In [71]:
promising.to_csv('./Outputs/Promising_Cust_09062021.csv', index=False)

clv_relevant.to_csv('./Outputs/High_MidHigh_CLV_Cust_09062021.csv', index=False)

## Experiments